## This notebook is for running `create_tables.py` and `etl.py` and checking if they work properly.
- You can create tables for preparing the database and build etl pipeline by runnig the cells below. 
- This notebook is also used to find in which query has errors.

In [2]:
# create_tables.py
import configparser
import psycopg2
from sql_queries import create_table_queries, drop_table_queries


def drop_tables(cur, conn):
    
    """
    Function to drop tables which already exist in the Redshift cluster. This excutes the queries in the variable drop_table_queries in the file sql_queries2.py.
    Parameters:
    - cur: cursor connection object
    - conn: connection object
    """
    
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()
        
def create_tables(cur, conn):
    
    """
    Function to create tables in the Redshift cluster. This excutes the queries in the variable create_table_queries in the file sql_queries2.py.
    Parameters:
    - cur: cursor connection object
    - conn: connection object
    """
    
    for query in create_table_queries:
        print(query)
        cur.execute(query)
        conn.commit()

def main():
    
    """
    Function to connect database in the Redshift cluster by reading configurations in the file dwh.cfg. 
    Additioanlly, this excutes the functions drop_tables(cur, conn) and create_tables(cur, conn).
    """
    
    config = configparser.ConfigParser()
    config.read('dwh.cfg')

    conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
    cur = conn.cursor()

    drop_tables(cur, conn)
    create_tables(cur, conn)

    conn.close()


if __name__ == "__main__":
    main()


CREATE TABLE staging_events(
    event_id INT IDENTITY(0,1) NOT NULL,
    artist_name VARCHAR(300),
    auth VARCHAR(50),
    user_first_name VARCHAR(300),
    user_gender  VARCHAR(1),
    item_in_session INTEGER,
    user_last_name VARCHAR(300),
    song_length DOUBLE PRECISION, 
    user_level VARCHAR(50),
    location VARCHAR(300),
    method VARCHAR(30),
    page VARCHAR(50),
    registration VARCHAR(50),
    session_id BIGINT,
    song_title VARCHAR(300),
    status INTEGER, 
    ts VARCHAR(50) NOT NULL,
    user_agent TEXT,
    user_id VARCHAR(100),
    PRIMARY KEY (event_id))


CREATE TABLE staging_songs(
    song_id VARCHAR(100) NOT NULL,
    num_songs INTEGER,
    artist_id VARCHAR(100) NOT NULL,
    artist_latitude DOUBLE PRECISION,
    artist_longitude DOUBLE PRECISION,
    artist_location VARCHAR(300),
    artist_name VARCHAR(300),
    title VARCHAR(300),
    duration DOUBLE PRECISION,
    year INTEGER,
    PRIMARY KEY (song_id))


CREATE TABLE songplays(
    songplay_id I

In [1]:
#etl.py
import configparser
import psycopg2
from sql_queries import copy_table_queries, insert_table_queries


def load_staging_tables(cur, conn):
     
    """
    Function to extract data from S3 bucket nad copy the contents of data to the stagin tables in the Redshift cluster. 
    This excutes the queries in the variable copy_table_queries in the file sql_queries2.py.
    Parameters:
    - cur: cursor connection object
    - conn: connection object
    """
    
    for query in copy_table_queries:
        cur.execute(query)
        conn.commit()


def insert_tables(cur, conn):
    
    """
    Function to transform data from the stagin tables into the fact and dimension tables in the Redshift cluster. 
    This excutes the queries in the variable insert_table_queries in the file sql_queries2.py.
    Parameters:
    - cur: cursor connection object
    - conn: connection object
    """
    
    for query in insert_table_queries:
        cur.execute(query)
        conn.commit()


def main():
    
    """
    Function to connect database in the Redshift cluster by reading configurations in the file dwh.cfg. 
    Additioanlly, this excutes the functions load_staging_tables(cur, conn) and insert_tables(cur, conn).
    """
    
    config = configparser.ConfigParser()
    config.read('dwh.cfg')

    conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
    cur = conn.cursor()
    
    load_staging_tables(cur, conn)
    insert_tables(cur, conn)

    conn.close()


if __name__ == "__main__":
    main()

copy staging_events 
                          from 's3://udacity-dend/log_data'
                          iam_role 'arn:aws:iam::315365539953:role/myRedshiftRole'
                          json 's3://udacity-dend/log_json_path.json';
                       
copy staging_songs 
                          from 's3://udacity-dend/song_data' 
                          iam_role 'arn:aws:iam::315365539953:role/myRedshiftRole'
                          json 'auto';
                      

INSERT INTO songplays (start_time, user_id, level, song_id, artist_id, session_id, location, user_agent) 
SELECT  
    DISTINCT TIMESTAMP 'epoch' + se.ts/1000 * interval '1 second' as start_time, 
    se.user_id, 
    se.user_level, 
    ss.song_id,
    ss.artist_id, 
    se.session_id,
    se.location, 
    se.user_agent
FROM staging_events se, staging_songs ss
WHERE se.page = 'NextSong' 
AND se.song_title = ss.title 
AND se.artist_name = ss.artist_name 
AND se.song_length = ss.duration


INSERT INTO users 